In [59]:
import pandas as pd
import json
import os
import datetime

In [60]:
data = pd.read_json('/home/rtst15/AutoCheckTimeStamping/f_rms/rms5min_5.json', lines=True)

In [61]:
type(data)

pandas.core.frame.DataFrame

In [62]:
# drop the column 3

data.drop(['_UID','SYSLOG_FACILITY','SYSLOG_IDENTIFIER','_GID','_COMM','_PID','_SOURCE_REALTIME_TIMESTAMP'], axis='columns', inplace=True)

In [63]:
# drop the column 2

data.drop(['_EXE', '_CMDLINE', '_CAP_EFFECTIVE','_SELINUX_CONTEXT','_AUDIT_SESSION','_AUDIT_LOGINUID', '_SYSTEMD_CGROUP','_SYSTEMD_UNIT', '_SYSTEMD_SLICE','PRIORITY' ], axis='columns', inplace=True)

In [64]:
# drop the column : monotonic stamp

data.drop(['__MONOTONIC_TIMESTAMP'], axis='columns', inplace=True)

In [65]:
# drop the column 1

data.drop(['__CURSOR', '_BOOT_ID', '_TRANSPORT', '_SYSTEMD_INVOCATION_ID','_MACHINE_ID', '_HOSTNAME','_SYSTEMD_OWNER_UID','_SYSTEMD_USER_SLICE'], axis= 'columns', inplace=True)

In [66]:
# split log message

null_list = []
cnt=0
for i in data['MESSAGE']:
    if(i.split(' ')[0] ==''):
        null_list.append(cnt)
    cnt+=1

In [67]:
# delete null message

null_list

data = data.drop(null_list)
data = data.reset_index(drop=True)

In [68]:
# use try-catch for detecting error

cnttt = 0
a = []
for i in data['MESSAGE']:
    try:
        i.split(' ')[1]
    except:
        print('this is error',cnttt)
        a.append(cnttt)
    cnttt+=1

In [69]:
# delete error index

data = data.drop(a)
data = data.reset_index(drop=True)

In [70]:
# delete log message

delete_list = []
cntt=0

for i in data['MESSAGE']:
    if i.split(' ')[1] != "rms":
        delete_list.append(cntt)
    cntt+=1

In [71]:
# delete log message 

data = data.drop(delete_list)
data = data.reset_index(drop=True)

In [72]:
# split log message for concat value

delete_msg = []
for i in data['MESSAGE']:
    delete_msg.append(i.split())

In [73]:
dm = pd.DataFrame(delete_msg)

In [74]:
# concat offset, freq, delay

rms_offset = pd.DataFrame(dm)[2]
std_freq = pd.DataFrame(dm)[6]
std_delay = pd.DataFrame(dm)[10]

df = pd.concat([rms_offset,std_freq,std_delay], axis=1, keys=['Rms Offset','Std Freq', 'Std Delay'])


In [75]:
# Convert Epoch time str:10

hdate = []
for i in data['__REALTIME_TIMESTAMP']:
    s = str(i)
    hdate.append(int(s[0:10]))
data['__REALTIME_TIMESTAMP'] = hdate

In [76]:
# datetime could get only integer as parameter

dtime = pd.to_datetime(hdate, unit='s')
minAndsec = dtime.strftime('%M:%S')

# Add column UTC

data['UTC'] = minAndsec

In [77]:
finalDf = pd.concat([data['__REALTIME_TIMESTAMP'],data['UTC'],df], axis=1)

In [78]:
# delete null row

finalDf= finalDf.dropna(axis=0)

In [79]:
# convert pandas data to int

finalDf = finalDf.astype({'__REALTIME_TIMESTAMP': 'int'})
finalDf = finalDf.astype({'Rms Offset': 'int'})
finalDf = finalDf.astype({'Std Freq': 'int'})
finalDf = finalDf.astype({'Std Delay': 'int'})

In [80]:
finalDf.describe()

,__REALTIME_TIMESTAMP,Rms Offset,Std Freq,Std Delay
count,1.450000e+02,1.450000e+02,145.000000,145.000000
mean,1.598233e+09,1.109234e+06,36275.531034,5734.896552
std,8.400397e+01,1.335623e+07,94.793566,34.595625
min,1.598233e+09,7.000000e+00,36109.000000,5655.000000
25%,1.598233e+09,4.400000e+01,36206.000000,5706.000000
50%,1.598233e+09,5.500000e+01,36263.000000,5731.000000
75%,1.598233e+09,6.900000e+01,36338.000000,5753.000000
max,1.598233e+09,1.608304e+08,36563.000000,5871.000000


In [81]:
# delete first value

finalDf = finalDf.drop([0])
finalDf = finalDf.reset_index(drop=True)

In [82]:
finalDf.to_csv('/home/rtst15/AutoCheckTimeStamping/f_csv_rms/rms_5min_5.csv')